In [1]:
"s-s".__contains__("-")

True

In [8]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch, os, numpy as np, pandas as pd
import re
# --- setup ---
imagedir = "images/real-1"
teach_df = pd.read_excel("teacher_transcript_PTJ.xlsx")
teach_df = teach_df.fillna("")

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").eval()
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


def clip_image_text_score(image_paths, text):
    """Compute average CLIP similarity between image(s) and all short sentences."""
    sims = []
    # Split long captions by punctuation or newline
    segments = re.split(r'[.!?]', text)
    segments = [seg.strip() for seg in segments if len(seg.strip()) > 0]

    for img_path in image_paths:
        if not os.path.exists(img_path):
            continue
        image = Image.open(img_path).convert("RGB")
        for seg in segments:
            inputs = clip_processor(text=[seg], images=image, return_tensors="pt", padding=True, truncation=True)
            with torch.no_grad():
                outputs = clip_model(**inputs)
                sims.append(outputs.logits_per_image[0, 0].item())

    # Take mean or max similarity across all sentence fragments
    return float(np.nanmax(sims)) if sims else float("nan")

# --- compute per-frame scores for each row ---
clip_scores = []

for _, row in teach_df.iterrows():
    row_scores = []
    for i in range(1, 7):
        img_path = os.path.join(imagedir, f"{i}.png")
        caption = row.get(f"{i}_image", "")
        score = clip_image_text_score([img_path], caption)
        row_scores.append(score)
    clip_scores.append(row_scores)

# --- make new DataFrame with same columns + scores ---
teach_df1 = teach_df.copy()
for i in range(6):
    teach_df1[f"clip_score_{i+1}"] = [s[i] for s in clip_scores]

teach_df1["clip_mean"] = [np.nanmean(s) for s in clip_scores]

# --- save or preview ---
print(teach_df1.head())
teach_df1.to_excel("teacher_clip_scores.xlsx", index=False)
print("Saved CLIP scores to selected_clip_scores.xlsx")

     Code Country Code  ID Sex CEFR Level  \
0   L_001           SG   1   F              
1  HW_001           SG   2   F              
2   J_001           SG   3   M              
3   L_002           SG   4   F              

                                          PTJ_PIC_xx  \
0   A few weeks ago. This boy was thinking to go ...   
1  A few weeks ago. This boy was dreaming about a...   
2  A few weeks ago, I wanted to go surfing and sw...   
3  A few weeks ago. The boy. Thought of going swi...   

                                             1_image  \
0   A few weeks ago. This boy was thinking to go ...   
1  A few weeks ago. This boy was dreaming about a...   
2  A few weeks ago, I wanted to go surfing and sw...   
3  A few weeks ago. The boy. Thought of going swi...   

                                             2_image  \
0             , but he realized that he got no money   
1  But then he realized that. He was absolutely b...   
2              However, I did not have enoug